<a href="https://colab.research.google.com/github/YigitBCesur/Time_Series/blob/main/Naive_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm

# Veri çekme
wmt_data = yf.download('WMT', start='2020-02-10', end='2025-02-10')
wmt_data = wmt_data[['Close']] # Keep only the 'Close' column as a DataFrame using double brackets

# 1. Durağanlık Kontrolü
def test_stationarity(timeseries):
    result = adfuller(timeseries, autolag='AIC')
    print('ADF İstatistiği:', result[0])
    print('p-değeri:', result[1])
    print('Kritik Değerler:', result[4])

test_stationarity(wmt_data['Close'])

# 2. Mevsimsellik ve Trend Analizi
decomposition = seasonal_decompose(wmt_data['Close'], model='additive', period=252)  # 252 işlem günü
fig = decomposition.plot()
plt.show()

# 3. Naive Forecast Uygulama (Farklı Gecikmeler)
for lag in [1, 5, 22]:  # 1 gün, 1 hafta, 1 ay
    wmt_data[f'Naive_Forecast_{lag}'] = wmt_data['Close'].shift(lag)

# 4. Hata Ölçütleri Hesaplama
for lag in [1, 5, 22]:
    mae = mean_absolute_error(wmt_data['Close'][lag:], wmt_data[f'Naive_Forecast_{lag}'][lag:])
    mse = mean_squared_error(wmt_data['Close'][lag:], wmt_data[f'Naive_Forecast_{lag}'][lag:])
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((wmt_data['Close'][lag:] - wmt_data[f'Naive_Forecast_{lag}'][lag:]) / wmt_data['Close'][lag:])) * 100
    print(f"\nNaive Forecast (Lag {lag}):")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAPE: {mape:.2f}%")

# 5. Model Değerlendirme Kriterleri
model = sm.tsa.SimpleExpSmoothing(wmt_data['Close']).fit()
aic = model.aic
bic = model.bic
print(f"\nModel Değerlendirme Kriterleri:")
print(f"AIC: {aic:.2f}")
print(f"BIC: {bic:.2f}")

# 6. Görselleştirme
plt.figure(figsize=(12, 6))
plt.plot(wmt_data['Close'], label='Gerçek Fiyat')
plt.plot(wmt_data['Naive_Forecast_1'], label='Naive Tahmin (1 gün)', linestyle='--')
plt.plot(wmt_data['Naive_Forecast_5'], label='Naive Tahmin (1 hafta)', linestyle=':')
plt.plot(wmt_data['Naive_Forecast_22'], label='Naive Tahmin (1 ay)', linestyle='-.')
plt.legend()
plt.title('Walmart Hisse Senedi Fiyatları ve Naive Tahminler')
plt.xlabel('Tarih')
plt.ylabel('Fiyat')
plt.show()




---



---



In [ ]:
!pip install yfinance pandas numpy matplotlib statsmodels scikit-learn

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm
import logging

# Loglama ayarları
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Veri indirme fonksiyonu
def download_data(ticker, start_date, end_date):
    """Hisse senedi verilerini indirir."""
    try:
        logging.info(f"{ticker} hisse senedi verileri indiriliyor...")
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty:
            raise ValueError("İndirilen veri boş. Lütfen tarih aralığını ve hisse senedini kontrol edin.")
        return data[['Close']]  # Sadece 'Close' sütununu döndür
    except Exception as e:
        logging.error(f"Veri indirme sırasında hata oluştu: {e}")
        return None

# Durağanlık testi fonksiyonu
def test_stationarity(timeseries):
    """Durağanlık testi yapar ve sonuçları yazdırır."""
    try:
        logging.info("Durağanlık testi yapılıyor...")
        result = adfuller(timeseries, autolag='AIC')
        print('\nADF İstatistiği:', result[0])
        print('p-değeri:', result[1])
        print('Kritik Değerler:', result[4])
    except Exception as e:
        logging.error(f"Durağanlık testi sırasında hata oluştu: {e}")

# Mevsimsellik ve trend analizi fonksiyonu
def seasonal_decomposition_analysis(timeseries, period=252):
    """Mevsimsellik ve trend analizi yapar."""
    try:
        logging.info("Mevsimsellik ve trend analizi yapılıyor...")
        decomposition = seasonal_decompose(timeseries, model='additive', period=period)
        fig = decomposition.plot()
        plt.show()
    except Exception as e:
        logging.error(f"Mevsimsellik ve trend analizi sırasında hata oluştu: {e}")

# Naive forecast hesaplama fonksiyonu
def naive_forecast(data, lags):
    """Naive forecast hesaplar ve veriye ekler."""
    try:
        logging.info("Naive forecast hesaplanıyor...")
        for lag in lags:
            data[f'Naive_Forecast_{lag}'] = data['Close'].shift(lag)
        return data
    except Exception as e:
        logging.error(f"Naive forecast hesaplama sırasında hata oluştu: {e}")
        return None

# Hata ölçütleri hesaplama fonksiyonu (DÜZELTİLDİ)
def calculate_error_metrics(data, lags):
    """Hata ölçütlerini hesaplar ve yazdırır."""
    try:
        logging.info("Hata ölçütleri hesaplanıyor...")
        for lag in lags:
            if f'Naive_Forecast_{lag}' not in data.columns:
                raise KeyError(f"Naive_Forecast_{lag} sütunu bulunamadı.")

            # NaN değerleri temizle
            valid_data = data.dropna(subset=['Close', f'Naive_Forecast_{lag}'])

            mae = mean_absolute_error(valid_data['Close'], valid_data[f'Naive_Forecast_{lag}'])
            mse = mean_squared_error(valid_data['Close'], valid_data[f'Naive_Forecast_{lag}'])
            rmse = np.sqrt(mse)
            mape = np.mean(np.abs((valid_data['Close'] - valid_data[f'Naive_Forecast_{lag}']) / valid_data['Close'])) * 100  # DÜZELTİLDİ

            print(f"\nNaive Forecast (Lag {lag}):")
            print(f"MAE: {mae:.2f}")
            print(f"MSE: {mse:.2f}")
            print(f"RMSE: {rmse:.2f}")
            print(f"MAPE: {mape:.2f}%")
    except KeyError as e:
        logging.error(f"Hata ölçütleri hesaplanırken sütun bulunamadı: {e}")
    except Exception as e:
        logging.error(f"Hata ölçütleri hesaplama sırasında hata oluştu: {e}")

# Model değerlendirme kriterleri fonksiyonu
def evaluate_model(data):
    """Model değerlendirme kriterlerini hesaplar."""
    try:
        logging.info("Model değerlendirme kriterleri hesaplanıyor...")
        model = sm.tsa.SimpleExpSmoothing(data['Close']).fit()
        aic = model.aic
        bic = model.bic
        print(f"\nModel Değerlendirme Kriterleri:")
        print(f"AIC: {aic:.2f}")
        print(f"BIC: {bic:.2f}")
    except Exception as e:
        logging.error(f"Model değerlendirme kriterleri hesaplanırken hata oluştu: {e}")

# Görselleştirme fonksiyonu
def plot_results(data, lags):
    """Sonuçları görselleştirir."""
    try:
        logging.info("Görselleştirme yapılıyor...")
        plt.figure(figsize=(12, 6))
        plt.plot(data['Close'], label='Gerçek Fiyat')
        for lag in lags:
            if f'Naive_Forecast_{lag}' in data.columns:
                plt.plot(data[f'Naive_Forecast_{lag}'], label=f'Naive Tahmin ({lag} gün)', linestyle='--')
            else:
                logging.warning(f"Naive_Forecast_{lag} sütunu bulunamadı. Görselleştirme atlandı.")
        plt.legend()
        plt.title('Walmart Hisse Senedi Fiyatları ve Naive Tahminler')
        plt.xlabel('Tarih')
        plt.ylabel('Fiyat')
        plt.show()
    except Exception as e:
        logging.error(f"Görselleştirme sırasında hata oluştu: {e}")

# Ana kod akışı
if __name__ == "__main__":
    # Veri indirme
    wmt_data = download_data('WMT', '2020-02-10', '2025-02-10')
    if wmt_data is None:
        exit()

    # Durağanlık testi
    test_stationarity(wmt_data['Close'])

    # Mevsimsellik ve trend analizi
    seasonal_decomposition_analysis(wmt_data['Close'])

    # Naive forecast hesaplama
    lags = [1, 5, 22]  # 1 gün, 1 hafta, 1 ay
    wmt_data = naive_forecast(wmt_data, lags)
    if wmt_data is None:
        exit()

    # Hata ölçütleri hesaplama
    calculate_error_metrics(wmt_data, lags)

    # Model değerlendirme kriterleri
    evaluate_model(wmt_data)

    # Görselleştirme
    plot_results(wmt_data, lags)



---



---

